In [1]:
import csv
from kafka import KafkaProducer
import requests
import json
import time

# Define your list of cities in Canada with unpredictable weather
cities = ['Toronto', 'Vancouver', 'Calgary', 'Montreal', 'Edmonton', 
          'Ottawa', 'Quebec City', 'Winnipeg', 'Halifax', 'Victoria', 
          'Regina', 'Saskatoon', 'St. John\'s', 'Hamilton', 'Kelowna', 
          'London', 'Thunder Bay', 'Charlottetown', 'Fredericton', 'Whitehorse']

# Kafka producer setup
producer = KafkaProducer(bootstrap_servers='localhost:9092')

# OpenWeather API key
openweather_api_key = '56718e7590a2f67ca64cfb140da39658'

# Columns to extract for CSV file
csv_columns = ['city', 'timestamp', 'temperature', 'humidity', 'weather_description', 'wind_speed', 'cloudiness']

# File to save the data
csv_file_path = 'weather_forecast.csv'

# Open CSV file for writing
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)

    # Write header to CSV file
    writer.writeheader()

    # Fetch one instance of data for each city and write to CSV
    for city in cities:
        url = f'http://api.openweathermap.org/data/2.5/weather?q={city},CA&appid={openweather_api_key}'
        response = requests.get(url)

        if response.status_code == 200:
            weather_data = response.json()

            # Extract necessary fields for CSV
            extracted_data = {
                'city': city,
                'timestamp': weather_data.get('dt', ''),
                'temperature': weather_data.get('main', {}).get('temp', ''),
                'humidity': weather_data.get('main', {}).get('humidity', ''),
                'weather_description': weather_data.get('weather', [{}])[0].get('description', ''),
                'wind_speed': weather_data.get('wind', {}).get('speed', ''),
                'cloudiness': weather_data.get('clouds', {}).get('all', '')
            }

            # Write data to CSV file
            writer.writerow(extracted_data)

            # Publish data to Kafka
            producer.send('weather_topic', json.dumps(extracted_data).encode('utf-8'))

            print(f"Data for {city} collected, written to CSV, and published to Kafka.")
        else:
            print(f"Failed to fetch data for {city}. Status code: {response.status_code}")

# Close the Kafka producer
producer.close()


Data for Toronto collected, written to CSV, and published to Kafka.
Data for Vancouver collected, written to CSV, and published to Kafka.
Data for Calgary collected, written to CSV, and published to Kafka.
Data for Montreal collected, written to CSV, and published to Kafka.
Data for Edmonton collected, written to CSV, and published to Kafka.
Data for Ottawa collected, written to CSV, and published to Kafka.
Data for Quebec City collected, written to CSV, and published to Kafka.
Data for Winnipeg collected, written to CSV, and published to Kafka.
Data for Halifax collected, written to CSV, and published to Kafka.
Data for Victoria collected, written to CSV, and published to Kafka.
Data for Regina collected, written to CSV, and published to Kafka.
Data for Saskatoon collected, written to CSV, and published to Kafka.
Data for St. John's collected, written to CSV, and published to Kafka.
Data for Hamilton collected, written to CSV, and published to Kafka.
Data for Kelowna collected, writte